# Adversarial Boosting

# Documentation

 - http://lightgbm.readthedocs.io/en/latest/
 - http://lightgbm.readthedocs.io/en/latest/Python-Intro.html
 - https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd
import lightgbm
import pickle
import json
import functools
from os import listdir
from os.path import isfile, join

from NILib import *

# Adversarial Boosting

In [2]:
def gen_adv_boosting_data(model, data, groups, num_atks=1):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    groups : grouping of same attacked instance 
    returns the new data matrix and new groups
    
    WARNING: currently works only for binary classification
    '''
    # score the datataset
    labels = data[:,-1]
    
    # check mispredictions
    predictions = model.predict(data[:,:-1]) # exclude labels
    matchings = labels * predictions
    
    # select original data + attacked instances
    new_selected = [] # id of selected instances
    new_groups   = []
    
    offset = 0
    for g in groups:
        if g==0:
            print ("Error !!!!")
        elif g==1:
            # there are no attacks, just add original
            new_selected += [offset]
            new_groups   += [1]
        else:
            # get a slice of the matching scores
            g_matchings = matchings[offset:offset+g]

            # most misclassified (smallest margin)
            # skip original
            #adv_instance = np.argmin(g_matchings[1:])+1
            adv_instances = np.argsort(g_matchings[1:])
            adv_instances = adv_instances[:num_atks]
            adv_instances += offset +1

            # add original and adversarial
            new_selected += [offset] + list(adv_instances)
            new_groups   += [1 + len(adv_instances)]
        
        offset += g
    
    new_dataset = data[new_selected,:]
    
    return new_dataset, new_groups

In [3]:
def extend_adv_boosting_model(train, valid, cat_fx, input_model=None, num_trees=1, params=None):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    returns the new model (is model modified inplace?)
    '''
    
    if cat_fx is None or len(cat_fx)==0:
        cat_fx = "auto"
        
    assert train.shape[1]==valid.shape[1], "Train/Valid Mismatch!"

    lgbm_train = lightgbm.Dataset(data=train[:,:-1], 
                                  label=train[:,-1],
                                  categorical_feature = cat_fx)
    
    lgbm_valid = lightgbm.Dataset(data=valid[:,:-1], 
                                  label=valid[:,-1],
                                  categorical_feature = cat_fx)
    
    lgbm_info = {}
    lgbm_model = lightgbm.train(params, lgbm_train, 
                                num_boost_round = num_trees, 
                                init_model = input_model,
                                fobj = optimize_log_loss, 
                                feval = avg_log_loss,
                                evals_result = lgbm_info,
                                valid_sets   = [lgbm_train, lgbm_valid], 
                                valid_names  = ['train', 'valid'],
                                verbose_eval=5)

    return lgbm_model, lgbm_info

In [4]:
def AdvBoosting(atk_train, atk_valid, trees, 
                 cat_fx,
                 params,
                 output_model_file,
                 partial_save=1000, 
                 adv_rounds=1):
    ''' 
    atk_data: full dataset including all valid attacks
    atk_groups: lenght of each attack set
    trees: total number of trees to be produced
    adv_rounds: adversarial instance injecting frequency
    '''
    # temp lgbm file
    temp = output_model_file+".tmp"
    
    # get groups and remove instance ids
    atk_groups = atk_train['instance_id'].value_counts().sort_index().values
    atk_valid_groups = atk_valid['instance_id'].value_counts().sort_index().values
    
    # get index of categorical features 
    cat_fx = np.where(atk_train.columns.isin(cat_fx))[0]
    cat_fx = list([int(x) for x in cat_fx])  
    # print ("CatFX:", atk_train.columns.values[cat_fx])

    # prepare data (avoiding pandas)
    atk_data   = atk_train.iloc[:,1:].values
    atk_valid  = atk_valid.iloc[:,1:].values
    cat_fx = [x - 1 for x in cat_fx]

    # train first trees
    original_ids = np.cumsum(atk_groups[:-1])
    original_ids = np.insert(original_ids, 0, 0)
    
    original_valid_ids = np.cumsum(atk_valid_groups[:-1])
    original_valid_ids = np.insert(original_valid_ids, 0, 0)
    
    model, model_info = extend_adv_boosting_model(atk_data[original_ids, :], 
                                                  atk_valid[original_valid_ids, :],
                                                  cat_fx=cat_fx,
                                                  input_model=None, 
                                                  num_trees=adv_rounds, 
                                                  params=params)
    
    best_model = model
    best_info = model_info
    best_loss = np.min(model_info['valid']['avg_binary_log_loss'])
    best_round = 1
        
    # train remaining trees
    for t in range(adv_rounds+1, trees+1, adv_rounds):
        # attack dataset
        adv_data, _       = gen_adv_boosting_data(model, atk_data, atk_groups)
        adv_valid_data, _ = gen_adv_boosting_data(model, atk_valid, atk_valid_groups)
        
        # train additional trees
        model.save_model(temp)
        model, model_info = extend_adv_boosting_model(adv_data, 
                                                      adv_valid_data,
                                                      cat_fx=cat_fx,
                                                      input_model=temp, 
                                                      num_trees=adv_rounds, 
                                                      params=params)

        if np.min(model_info['valid']['avg_binary_log_loss']) < best_loss:
            best_model = model
            best_info  = model_info
            best_loss  = np.min(model_info['valid']['avg_binary_log_loss'])
            best_round = t
        
        # save partial model
        if t % partial_save == 0 and t != trees:
            partial_filename = "{:s}_T{:d}-of-{:d}_S{:04d}_L{:d}.model.tmp".format(output_model_file, 
                                                                                   t, 
                                                                                   trees, 
                                                                                   int(params['learning_rate'] * 1000),
                                                                                   params['num_leaves']
                                                                                  )
            
            print("Save partial model to {}".format(partial_filename))
            model.save_model(filename=partial_filename)
            
    
    return model, model_info, best_loss, best_round

In [5]:
def train_adversarial_boosting(train_file, valid_file, test_file, output_model_file):
    
    exp = pd.DataFrame(columns=['num_trees', 'learning_rate', 'num_leaves', 'best_round', 'avg_binary_log_loss'])
    
    # load train/valid/test
    train, valid, test, cat_fx = load_atk_train_valid_test(train_file, valid_file, test_file)
    assert "instance_id" in train.columns.values, "Wrong training set file for GBDT"

    for num_trees in [200]:
        for learning_rate in [0.1]: #[0.01, 0.05, 0.1]:
            for num_leaves in [16]: #[8, 16, 24]:
                      
                lgbm_params = { 'learning_rate': learning_rate, 
                                'num_leaves': num_leaves} 
                
                lgbm_model, lgbm_info, best_loss, best_valid_iter = AdvBoosting(train,
                                                    valid,
                                                    trees=num_trees, 
                                                    cat_fx = cat_fx, 
                                                    output_model_file=output_model_file, 
                                                    adv_rounds=1,
                                                    params=lgbm_params)
                

                ####
                #best_model = lightgbm.Booster(model_file=model_file_name)
                atk_valid_groups = valid['instance_id'].value_counts().sort_index().values
                cat_fx = np.where(valid.columns.isin(cat_fx))[0]
                cat_fx = list([int(x) for x in cat_fx])  

                atk_valid  = valid.iloc[:,1:].values
                cat_fx = [x - 1 for x in cat_fx]
                
                original_valid_ids = np.cumsum(atk_valid_groups[:-1])
                original_valid_ids = np.insert(original_valid_ids, 0, 0)

                lgbm_valid = lightgbm.Dataset(data=atk_valid[original_valid_ids,:-1], 
                                              label=atk_valid[original_valid_ids,-1],
                                              categorical_feature = cat_fx)
                
                
                lgbm_valid_att = lightgbm.Dataset(data=atk_valid[:,:-1], 
                                              label=atk_valid[:,-1],
                                              categorical_feature = cat_fx)
                
                print ("Check valid score without attacks:", avg_log_loss(preds=lgbm_model.predict(atk_valid[original_valid_ids,:-1]),
                                                  train_data=lgbm_valid))
                
                print ("Check valid score with attacks:", avg_log_loss(preds=lgbm_model.predict(atk_valid[:,:-1]),
                                                                       train_data=lgbm_valid_att))

                
                ####
                # update experimental results
                exp = exp.append({'num_trees': num_trees, 
                                  'learning_rate':learning_rate,
                                  'num_leaves':num_leaves, 
                                  'best_round':best_valid_iter, 
                                  'avg_binary_log_loss':best_loss},
                                 ignore_index=True)
        
        # save file
        model_file_name = "{:s}_T{:d}_S{:04d}_L{:d}_R{:d}.model".format(output_model_file,
                                                                                num_trees,
                                                                                int(learning_rate*1000),
                                                                                num_leaves,
                                                                                best_valid_iter
                                                                               )
        lgbm_model.save_model(model_file_name)
        print ("Model saved to", model_file_name)
                
    return exp

In [6]:
# enable/disable
if True:
    for B in [150]: #[5, 15, 150, 300]:

        experiments = train_adversarial_boosting ( "../data/census/train_B{:d}.csv.bz2".format(B),
                                                   "../data/census/valid_B{:d}.csv.bz2".format(B),
                                                   "../data/census/test_B{:d}.csv.bz2".format(B),
                                                   "../out/models/adv_boosting_census_B{:d}".format(B))  

        experiments.to_csv('../out/models/adv_boosting_census_B{:d}.csv'.format(B), index=False)

        print (experiments)

Loading pre-processed files...


/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[5]	train's avg_binary_log_loss: 0.473641	valid's avg_binary_log_loss: 0.473848
[10]	train's avg_binary_log_loss: 0.375873	valid's avg_binary_log_loss: 0.376209
[15]	train's avg_binary_log_loss: 0.327713	valid's avg_binary_log_loss: 0.327717
[20]	train's avg_binary_log_loss: 0.302276	valid's avg_binary_log_loss: 0.302374
[25]	train's avg_binary_log_loss: 0.288803	valid's avg_binary_log_loss: 0.288819
[30]	train's avg_binary_log_loss: 0.281296	valid's avg_binary_log_loss: 0.281551
[35]	train's avg_binary_log_loss: 0.275863	valid's avg_binary_log_loss: 0.276531
[40]	train's avg_binary_log_loss: 0.272852	valid's avg_binary_log_loss: 0.2734
[45]	train's avg_binary_log_loss: 0.270462	valid's avg_binary_log_loss: 0.271391
[50]	train's avg_binary_log_loss: 0.268955	valid's avg_binary_log_loss: 0.270248
[55]	train's avg_binary_log_loss: 0.263705	valid's avg_binary_log_loss: 0.265344
[60]	train's avg_binary_log_loss: 0.262287	valid's avg_binary_log_loss: 0.264185
[65]	train's avg_binary_log_los

**some old results**

    Check valid score without attacks: ('avg_binary_log_loss', 0.3237495543276321, False)
    Check valid score with attacks: ('avg_binary_log_loss', 0.03632538905070899, False)
    Model saved to ../out/models/adv_boosting_census_B150_T200_S0100_L16_R200.model
       num_trees  learning_rate  num_leaves  best_round  avg_binary_log_loss
    0      200.0            0.1        16.0       200.0             0.224304